In [4]:
import pandas as pd 
import numpy as np
from scipy.stats import pearsonr
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import os
from multiprocessing import Pool
#import deepgraph as dg

Data

https://www.synapse.org/#!Synapse:syn2812961 

https://www.kaggle.com/murats/gene-expression-cancer-rnaseq?fbclid=IwAR3Mj1xpvD_sBBwn6ivsPf04pvWIm7n3QCKwSdb_bmxhFL2iEXvCGIParIs

In [6]:
LGG = pd.read_csv("unc.edu_LGG_IlluminaHiSeq_RNASeqV2.geneExp.tsv", sep='\t')
LUCS = pd.read_csv("unc.edu_LUSC_IlluminaHiSeq_RNASeqV2.geneExp.tsv", sep='\t')
PRAD = pd.read_csv("unc.edu_PRAD_IlluminaHiSeq_RNASeqV2.geneExp.tsv", sep = '\t')

FileNotFoundError: [Errno 2] File b'unc.edu_LUSC_IlluminaHiSeq_RNASeqV2.geneExp.tsv' does not exist: b'unc.edu_LUSC_IlluminaHiSeq_RNASeqV2.geneExp.tsv'

Rows: Genes \
Columns: Patients

In [ ]:
LGG.head(5)

In [ ]:
LGG.shape

How to build a coexpression matrix?

https://link.springer.com/protocol/10.1007%2F978-1-4939-7747-5_21

In [ ]:
# Pearson Correlation
def calculate_pearson(data, length):
    '''
    Input: Data sequence
    Output: Person matrix
    '''
    gene_ids = data['gene_id'][:length]
    n = len(gene_ids)
    pmatrix = np.empty((n, n))
    
    for i in range(n):
        for j in range(n):
            gene1 = list(data.iloc[i, :].values)[1:][:length]
            gene2 = list(data.iloc[j, :].values)[1:][:length]
            # note: if either one of gene is all zeros, r and p are nan
            r, p= pearsonr(gene1, gene2)
            pmatrix[i][j] = r
    return pmatrix

In [3]:
# multi-processing for pearson r computation
# https://deepgraph.readthedocs.io/en/latest/tutorials/pairwise_correlations.html?fbclid=IwAR019mCREkqAlWLD_tDtVn_3LLj-NkAIyJJZMpr2_nYh_sEtDV404Eac5Xc

# tranpose to line up with demonstration in the link above
LGG_T = np.array(LGG.transpose().values)

# compute ranked variables for Spearman's correlation coefficients
LGG_T = LGG_T.argsort(axis=1).argsort(axis=1)
# whiten variables for fast parallel computation later on
LGG_T = (LGG_T - LGG_T.mean(axis=1, keepdims=True))/ LGG_T.std(axis=1, keepdims=True)

# save in binary format
np.save("LGG_T", LGG_T)

In [4]:
# parameters (change these to control RAM usage)
step_size = 1e5
n_processes = 1e4
n_features, n_samples = LGG.shape

# load data as memory-map
LGG_T = np.load("LGG_T.npy", mmap_mode="r")

# create node table that stores references to the mem-mapped samples
v = pd.DataFrame({"index":range(LGG_T.shape[0])})

# connector function to compute pairwise pearson correlations
def corr(index_s, index_t):
    features_s = LGG_T[index_s]
    features_t = LGG_T[index_t]
    corr = np.einsum("ij,ij->i", features_s, features_t) / n_samples
    return corr

# index array for parallelization
pos_array = np.array(np.linspace(0, n_features*(n_features-1)//2, n_processes), dtype=int)

# parallel computation
def create_ei(i):
        
    from_pos = pos_array[i]
    to_pos = pos_array[i+1]
    
    # initiate DeepGraph
    g = dg.DeepGraph(v)
    
    # crate edges
    g.create_edges(connectors=corr, step_size=step_size,
                  from_pos = from_pos, to_pos=to_pos)
    
    # store edge table
    g.e.to_pickle("tmp/correlations/{}.pickle".format(str(i).zfill(3)))
    
# computation
#os.makedirs("tmp/correlations", exist_ok=True)
indices = np.arange(0, n_processes-1, dtype=int)
p = Pool()
for _ in p.imap_unordered(create_ei, indices):
    pass
LGG_T.close()
p.close()

AssertionError: the given to_pos parameter is too large, 168634 > g.n*(g.n-1)/2=142845.0

In [ ]:
# store correlation values
files = os.listdir('tmp/correlations/')
files.sort()
store = pd.HDFStore('e.h5', mode='w')
for f in files:
    et = pd.read_pickle('tmp/correlations/{}'.format(f))
    store.append('e', et, format='t', data_columns=True, index=False)
store.close()

# load correlation table
e = pd.read_hdf('e.h5')
print(e)

In [ ]:
LUCS_T = np.array(LUCS.transpose().values)
PRAD_T = np.array(PRAD.transpose().values)

In [ ]:
lgg_num_genes, lgg_num_patients = LGG.shape
LGG_matrix = calculate_pearson(LGG, lgg_num_genes)
lucs_num_genes, lucs_num_patients = LUCS.shape
LUCS_matrix = calculate_pearson(LUCS, lucs_num_genes)
prad_num_genes, prad_num_patients = PRAD.shape
PRAD_matrix = calculate_pearson(PRAD, prad_num_genes)

In [ ]:
LGG_matrix.shape

In [ ]:
def write_to_file(filename, data):
    outfile = open(filename, 'wb')
    pickle.dump(data, outfile)
    outfile.close()


def load_pkl(filename):
    file = open(filename, 'rb')
    object_file = pickle.load(file)
    file.close()
    return object_file

In [ ]:
write_to_file("lgg_pearson_matrix.p", LGG_matrix)
write_to_file("lucs_pearson_matrix.p", LUCS_matrix)
write_to_file("prad_pearson_matrix.p", PRAD_matrix)

In [ ]:
# check for symmetric
def is_symmetric(matrix):
    n = len(matrix)
    for i in range(n):
        for j in range(n):
            if matrix[i][j] != matrix[j][i]:
                # can be nan
                if not np.isnan(matrix[i][j]) or not np.isnan(matrix[i][j]):
                    return False
    return True

In [ ]:
is_symmetric(LGG_matrix)

In [ ]:
def build_graph(data, adj_matrix, actual_name):
    # undirected, weighted graph
    G = nx.Graph()
    for i in range(len(adj_matrix)):
        if actual_name:
            G.add_node(data["gene_id"].iloc[i])
        else:
            G.add_node(i)
        for j in range(i+1, len(adj_matrix)):
            if not np.isnan(adj_matrix[i][j]): 
                if actual_name:
                    G.add_edge(data["gene_id"].iloc[i], data["gene_id"].iloc[j], weight=adj_matrix[i][j])
                else:
                    G.add_edge(i, j, weigt=adj_matrix[i][j])
    return G
                                        

In [ ]:
G = build_graph(LGG, LGG_matrix, False)

In [ ]:
# test to check graph is correct
print ("Number of nodes: ", nx.number_of_nodes(G) == len(LGG_matrix))
print ("Number of selfloop: ", nx.number_of_selfloops(G))
print (G.get_edge_data(0,1), LGG_matrix[0][1])
print (G.get_edge_data(2, 4), LGG_matrix[2][4])

In [ ]:
print ("nodes number: {}, edges number: {}".format(nx.number_of_nodes(G), nx.number_of_edges(G)))
print ("Number of connected components: ", nx.number_connected_components(G))

In [ ]:
def analyze_connected_component(graph):
    largest_connected_component = max(nx.connected_component_subgraphs(graph), key=len)
    num_nodes = len(largest_connected_component.nodes)
    degree = largest_connected_component.degree() 
    return largest_connected_component, num_nodes, degree

In [ ]:
largest_cc, num_nodes, degree = analyze_connected_component(G)

In [ ]:
print ("largest cc num of nodes: ", num_nodes)

In [ ]:
# centrality
def centrality(G):
    betweenness = nx.betweenness_centrality(G, normalized=True)
    eigenvec = nx.eigenvector_centrality_numpy(G, weight="weight")
    degree = nx.degree_centrality(G)
    closeness = nx.closeness_centrality(G)
    return degree, betweenness, closeness, eigenvec

In [ ]:
degree, betweenness, closeness, eigenvec = centrality(G)

In [ ]:
# build a dataframe for genes and their centrality values
def build_dataframe(data, inputs, actual_name):
    #inputs is dictionaries within dictionaries
    # outer dict: key= centrality type, value: dictionary of computed values for all genes
    # inner dict: key= gene id (index or actual name), value: centrality value for the gene id
    centrality_df = data[["gene_id"]][:100].copy()
    for centrality in inputs.keys():
        values = []
        for i, gene_id in data["gene_id"][:100].iteritems():
            if actual_name:
                values.append(inputs[centrality][gene_id])
            else:
                values.append(inputs[centrality][i])
        centrality_df[centrality] = pd.DataFrame(values)
    return centrality_df

In [ ]:
inputs = {"degree": degree, "betweenness": betweenness, "closeness": closeness, "eigenvector": eigenvec}
centrality_df = build_dataframe(LGG, inputs, False)
centrality_df.shape

In [ ]:
centrality_df.tail(5)